# Tarea 4
- Paula Cintron 20160090
miercoles 14 de agosto 2019

wine quality data

In [166]:
import torch
import torch.nn as nn
import numpy as np
from numpy import genfromtxt
import torch.optim as optim

In [167]:
# Import data, separado por punto y coma
wine = genfromtxt("wine.csv", delimiter=';')

In [168]:
wine.shape

(4899, 12)

In [169]:
wine.view(type=np.matrix)

matrix([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
        [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
        ...,
        [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
        [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
        [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]])

In [170]:
# un poco de data cleaning
# eliminar la primera fila porque son los títulos de la tabla 
wine2= np.delete(wine,0,0)

In [171]:
wine2.view()

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]])

In [172]:
# seleccionar solo la comlumna de calidad
calidad = wine2[:,11]
calidad

array([6., 6., 6., ..., 6., 7., 6.])

In [173]:
# eliminar la columna calidad del resto de la data
wine2= np.delete(wine2,11,1)

In [174]:
# Todas las variables estan listas para ser utilizadas en la red neuronal
wine2

array([[ 7.  ,  0.27,  0.36, ...,  3.  ,  0.45,  8.8 ],
       [ 6.3 ,  0.3 ,  0.34, ...,  3.3 ,  0.49,  9.5 ],
       [ 8.1 ,  0.28,  0.4 , ...,  3.26,  0.44, 10.1 ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  2.99,  0.46,  9.4 ],
       [ 5.5 ,  0.29,  0.3 , ...,  3.34,  0.38, 12.8 ],
       [ 6.  ,  0.21,  0.38, ...,  3.26,  0.32, 11.8 ]])

In [175]:
# convertir la data en tensores
features= torch.tensor(wine2)
y = torch.tensor(calidad)

In [176]:
# Corroborar que si son tensores
torch.is_tensor(y)

True

In [177]:
torch.is_tensor(features)

True

In [178]:
y.shape

torch.Size([4898])

In [179]:
features.shape

torch.Size([4898, 11])

In [196]:
t_c = torch.tensor(features).unsqueeze(1) # Agregamos una dimension para tener B x N_inputs
t_u = torch.tensor(y).unsqueeze(1) # Agregamos una dimension para tener B x N_inputs

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [197]:
# Definiendo el training loop

def training_loop(model, n_epochs, optimizer, loss_fn, train_x, val_x, train_y, val_y):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_x) # ya no tenemos que pasar los params
        train_loss = loss_fn(train_t_p, train_y)
        
        with torch.no_grad(): # todos los args requires_grad=False
            val_t_p = model(val_x)
            val_loss = loss_fn(val_t_p, val_y)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss}, Validation loss {val_loss}")

In [198]:
# Armando el modelo NN

seq_model= nn.Sequential(nn.Linear(13,2), 
                         nn.Tanh(), 
                         nn.Linear(13,1))

In [199]:
[param.size() for param in seq_model.parameters()]

[torch.Size([2, 13]), torch.Size([2]), torch.Size([1, 13]), torch.Size([1])]

In [200]:
for name, param in seq_model.named_parameters(): print (name, param.size())

0.weight torch.Size([2, 13])
0.bias torch.Size([2])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [201]:
#from collections import OrderedDict

#named_seq_model = nn.Sequential(OrderedDict([
       # ('hidden_linear', nn.Linear(1, 8)),
        #('hidden_activation', nn.Tanh()),
        #('output_linear', nn.Linear(8, 1))
#]))

#seq_model

In [202]:
#for name, param in named_seq_model.named_parameters():
    #print(name, param.size())

In [203]:
#named_seq_model.output_linear.bias

In [204]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(), # Ya no estamos usando nuestra loss function hecha a mano
    train_x = train_t_u,
    val_x = val_t_u,
    train_y = train_t_c,
    val_y = val_t_c)

print('output', seq_model(val_t_un))
print('answer', val_t_c)
print('hidden', seq_model.hidden_linear.weight.grad)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'